# LangChain prompts常用的提示词模板
## PromptTemplate（字符串提示模板）
适用场景：
- 用于文本补全模型，输入是纯文本（单字符串）。
- 适用于简单的任务，例如生成一段文本、回答问题或执行指令。
特点：
- 输入变量插值：通过 {} 占位符动态替换变量。
- 模板格式：支持 f-string。
- 输出形式：生成一个完整的字符串作为模型输入。
示例：
```python
template_prompt = PromptTemplate.from_template("今天{something}真不错")
print(template_prompt) # input_variables=['something'] input_types={} partial_variables={} template='今天{something}真不错'

formatted_prompt = template_prompt.format(something="天气")
print(formatted_prompt) # 今天天气真不错
```

## ChatPromptTemplate（聊天提示模板）
适用场景：
- 用于聊天模型（如 ChatGPT / 通义千问等），输入是多轮对话的消息列表（SystemMessage、HumanMessage、AIMessage 等）。
- 适用于需要模拟多轮对话或角色扮演的场景。
特点：
- 多消息类型支持：可以组合系统指令、用户输入和助手回复。
- 消息格式化：生成结构化的消息列表，供聊天模型处理。
- 灵活性：支持动态替换变量（如 SystemMessage 中的占位符）。
示例：
```python
system_message = "你是一位{role}专家，擅长回答{domain}领域的问题。"
human_message = "{question}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_message)
])

# 格式化消息
formatted_messages = chat_prompt.format_messages(
    role="技术",
    domain="Web开发",
    question="如何构建一个基于Vue的前端应用？"
)
print(formatted_messages)
"""
[
  SystemMessage(content='你是一位技术专家，擅长回答Web开发领域的问题。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='如何构建一个基于Vue的前端应用？', additional_kwargs={}, response_metadata={})
]
"""
```

> python基础知识扩充：
  这里的 ("system", system_message)是 python 的 tuple 类型
  Tuple（元组）：
    - 有序：元素按顺序排列，支持索引和切片操作。
    - 不可变：创建后无法修改（不能增删改元素）。
    - 定义方式：使用圆括号 () 或直接逗号分隔元素。
  Array（数组）：相当于 JS 中的 Array
    - 可变：可以动态添加、删除或修改元素。
    - 有序：元素按顺序排列，支持索引和切片。
  Dict（字典）：相当于 JS 中的 Object
    - 无序：元素（键值对）没有固定顺序。
    - 可变：可以动态添加、删除或修改键值对。
    - 定义方式：使用花括号 {}，键值对用冒号 : 分隔。

知识扩充：ChatMessagePromptTemplate可以结合 ChatPromptTemplate使用，同时对提示词模板和消息体进行抽象和复用：
```python
system_template = ChatMessagePromptTemplate.from_template(
    template="你是一位{role}专家，擅长回答{domain}领域的问题。",
    role="system",
)

human_template = ChatMessagePromptTemplate.from_template(
    template="用户问题：{question}",
    role="human",
)

chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
])

messages = chat_prompt.format_messages(
    role="技术",
    domain="Web开发",
    question="如何构建一个基于Vue的前端应用？"
)

print(messages)

"""
[
  ChatMessage(content='你是一位技术专家，擅长回答Web开发领域的问题。', additional_kwargs={}, response_metadata={}, role='system'),
  ChatMessage(content='用户问题：如何构建一个基于Vue的前端应用？', additional_kwargs={}, response_metadata={}, role='human')
]
"""
```

## FewShotPromptTemplate（少样本提示模板）
适用场景：
- 用于少样本学习（Few-Shot Learning），在提示中包含示例（Examples），帮助模型理解任务。
- 适用于复杂任务（如翻译、分类、推理），需要通过示例引导模型行为。
特点：
- 示例嵌入：通过 examples 参数提供示例输入和输出。
- 动态示例选择：支持 ExampleSelector 动态选择最相关的示例。
- 模板格式：通常包含前缀（Prefix）、示例（Examples）和后缀（Suffix）。
示例：
```python
# 定义示例模板
example_template = "输入: {input}\n输出: {output}"

# 创建示例
examples = [
    {"input": "将'Hello'翻译成中文", "output": "你好"},
    {"input": "将'Goodbye'翻译成中文", "output": "再见"},
]

# 创建少样本模板
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=PromptTemplate.from_template(example_template),
    prefix="请将以下英文翻译成中文：",
    suffix="输入: {text}\n输出:",
    input_variables=["text"]
)

# 格式化提示
formatted_prompt = few_shot_prompt.format(text="Thank you")
print(formatted_prompt)

"""
请将以下英文翻译成中文：

输入: 将'Hello'翻译成中文
输出: 你好

输入: 将'Goodbye'翻译成中文
输出: 再见

输入: Thank you
输出:
"""
```

# 总结对比
## 常用模板类特性和使用场景对比
|子类	|适用模型类型	|输入类型	|主要用途|
| ---- | ---- | ---- | ----|
|PromptTemplate|	文本补全模型|	单字符串|	生成单轮文本任务的提示|
|ChatPromptTemplate|	聊天模型|	多消息列表|	模拟多轮对话或角色扮演|
|FewShotPromptTemplate|	所有模型|	包含示例的模板|	通过示例引导模型完成复杂任务|

## 常用提示词模板类的继承关系
![img_3.png](./images/img_3.png)